### Description
Author: T. Majidzadeh

Date Created: March 9, 2025

Date Updated: March 9, 2025

Purpose: Test difference-in-difference regression for Zillow rent indices, 2015-2019, treatment in Dec 2017. Prototype version assumes "affected" metros are those with at least 35% post-merger penetration rate and at least 10% share gain from the merger.

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import json
import os
import re

In [3]:
paths = {
    "zillow_raw": "..\\data\\zillow_data_raw\\",
    "zillow_reg": "..\\data\\zillow_reg_data\\"
}

In [4]:
zillow_reg = pd.read_pickle(paths['zillow_reg']+'zillow_data_reg_proto_20250309.pkl')

In [5]:
zillow_reg['AffectedCityTime'] = zillow_reg \
    .apply(lambda x: x.AffectedCity * x.AffectedTime, axis=1)
zillow_reg['Year'] = zillow_reg['Year'].astype(str)
zillow_reg['Month'] = zillow_reg['Month'].astype(str)

In [6]:
zillow_reg = pd \
    .get_dummies(zillow_reg, columns=['RegionName', 'Year', 'Month'], drop_first = True)
zillow_reg

,StateName,Year-Month,ZORI,AffectedCity,AffectedTime,TimeTrend,ZORI-Lag1,AffectedCityTime,"RegionName_Boston, MA","RegionName_Chicago, IL",...,Month_11,Month_12,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9
1,NY,2015-01-31,2209.959116,0,0,0,NaN,0,False,False,...,False,False,False,False,False,False,False,False,False,False
2,CA,2015-01-31,1684.237669,0,0,0,NaN,0,False,False,...,False,False,False,False,False,False,False,False,False,False
3,IL,2015-01-31,1366.441879,0,0,0,NaN,0,False,True,...,False,False,False,False,False,False,False,False,False,False
4,TX,2015-01-31,1024.638892,1,0,0,NaN,0,False,False,...,False,False,False,False,False,False,False,False,False,False
5,TX,2015-01-31,1138.420831,0,0,0,NaN,0,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29103,MN,2019-12-31,1389.784723,0,1,59,1386.426145,0,False,False,...,False,True,False,False,False,False,False,False,False,False
29104,CA,2019-12-31,2018.979007,0,1,59,2011.736617,0,False,False,...,False,True,False,False,False,False,False,False,False,False
29105,FL,2019-12-31,1236.871601,1,1,59,1232.537853,1,False,False,...,False,True,False,False,False,False,False,False,False,False
29106,CO,2019-12-31,1538.059242,1,1,59,1539.900116,1,False,False,...,False,True,False,False,False,False,False,False,False,False


In [31]:
yvar = 'ZORI'
xvars = [
    'TimeTrend', 
    'AffectedCity', 
    'AffectedCityTime'
]
city_fe = [col for col in zillow_reg.columns if col.startswith('RegionName')]
year_fe = [col for col in zillow_reg.columns if (col.startswith('Year')) & (col != "Year-Month")]
month_fe = [col for col in zillow_reg.columns if col.startswith('Month')]
zillow_reg_base=zillow_reg[[yvar]+xvars+city_fe+year_fe+month_fe].dropna()

Base Model

In [34]:
y = np.asarray(zillow_reg_base[yvar])
X = sm.add_constant(zillow_reg_base[xvars+city_fe+year_fe+month_fe], prepend=True) * 1


In [36]:
mod = sm.OLS(y, X, hasconst=True)
res = mod.fit(cov_type='HC3')
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.526e+05
Date:                Sun, 09 Mar 2025   Prob (F-statistic):               0.00
Time:                        13:50:39   Log-Likelihood:                -6039.2
No. Observations:                1200   AIC:                         1.215e+04
Df Residuals:                    1164   BIC:                         1.233e+04
Df Model:                          35                                         
Covariance Type:                  HC3                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
